In [1]:
# Parameters
autoregression = True
data_file = "experiments/pilot_random1_player_round_slim.csv"
device = "cpu"
experiment_names = ["trail_rounds_2", "random_1"]
fraction_training = 1.0
job_id = "architecture_node+edge+rnn"
labels = {"architecture": "node+edge+rnn"}
mask_name = "contribution_valid"
model_args = {
    "add_edge_model": True,
    "add_global_model": False,
    "add_rnn": True,
    "hidden_size": 5,
    "x_encoding": [
        {"encoding": "numeric", "n_levels": 21, "name": "contribution"},
        {"encoding": "numeric", "n_levels": 21, "name": "prev_contribution"},
        # {"encoding": "numeric", "n_levels": 31, "name": "prev_punishment"},
        # {"etype": "bool", "name": "contribution_valid"},
    ],
    "y_levels": 21,
    "y_name": "contribution",
}
model_name = "graph"
n_contributions = 21
n_cross_val = None
n_player = 4
n_punishments = 31
optimizer_args = {"lr": 0.0003, "weight_decay": 1e-05}
seed = 71826387
shuffle_features = ["prev_punishment", "prev_contribution", "contribution_valid", "contribution", "contribution_masked"]
train_args = {
    "epochs": 30,
    "batch_size": 20,
    "clamp_grad": 1,
    "eval_period": 20,
    "l1_entropy": 0
}
max_predicted = 4
min_predicted = 4
output_dir = "../../notebooks/artificial_humans/graph/node+edge+rnn"
basedir = "../.."


In [2]:
#   shuffle_features:
#     - contribution
#     - prev_punishment
#     - contribution_valid
#     - prev_punishment_valid
#     - punishment_masked

In [3]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import random
import torch as th
from aimanager.generic.data import create_torch_data, get_cross_validations
from aimanager.artificial_humans import AH_MODELS
from aimanager.artificial_humans.evaluation import eval_model, Recorder, create_confusion_matrix
from aimanager.utils.utils import make_dir
# from aimanager.generic.graph_encode import create_fully_connected
from aimanager.generic.graph import Encoder
# from torch_geometric.loader import DataLoader
from itertools import permutations


if "data_dir" in locals():
    output_dir = data_dir

if autoregression:
    if "min_predicted" not in locals():
        min_predicted = 1
    if "max_predicted" not in locals():
        max_predicted = n_player

model_dir = os.path.join(output_dir, 'model')
conf_dir = os.path.join(output_dir, 'confusion_matrix')
make_dir(model_dir)
make_dir(conf_dir)

model_path = os.path.join(model_dir, f"{job_id}.pt")
conf_path = os.path.join(conf_dir, f"{job_id}.parquet")

th.random.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)


In [4]:

df = pd.read_csv(os.path.join(basedir, data_file))

df = df[df['experiment_name'].isin(experiment_names)]

data, default_values = create_torch_data(df)

In [5]:
rec = Recorder()

th_device = th.device(device)


if autoregression:
    # the training mask defines which agents are predicted (true: agent will be predicted)
    # correspondingly we mask inputs for these agents
    # here we define the base patter (i.e. the number of agents predicted)
    pattern = [
        [True] * i + [False] * (n_player - i) for i in range(min_predicted, max_predicted + 1)
    ]
    test_pattern = [
        [True] * i + [False] * (n_player - i) for i in range(1, n_player + 1)
    ]
else:
    pattern = [[True] * n_player]
    test_pattern = [[True] * n_player]

# we create all possible permutations of the training mask
training_mask_pattern = list(set([pp for p in pattern for pp in permutations(p)]))
training_mask_pattern = th.tensor(training_mask_pattern, dtype=th.bool)

test_mask_pattern = list(set([pp for p in test_pattern for pp in permutations(p)]))
test_mask_pattern = th.tensor(test_mask_pattern, dtype=th.bool)


def shuffle_feature(data, feature_name):
    data = {**data}
    data[feature_name] = data[feature_name][th.randperm(len(data[feature_name]))]
    return data


def batch_loader(data, batch_size):
    n = len(data["contribution"])
    all_idx = np.arange(n)
    all_idx = np.random.permutation(all_idx)
    n_batch = int(np.ceil(n / batch_size))
    for i in range(n_batch):
        batch_idx = all_idx[i * batch_size : (i + 1) * batch_size]
        if len(batch_idx) != batch_size:
            continue
        batch = {k: v[batch_idx] for k, v in data.items()}
        yield batch

def mask_data(data, mask, target, default_values):
    data = {**data}
    data[target + '_masked'] = data[target].clone()
    if mask.shape[0] != data[target + '_masked'].shape[0]:
        mask = mask.repeat(data[target + '_masked'].shape[0], 1)
    data[target + '_masked'][mask] = default_values[target]
    return data

def apply_mask_pattern(data, mask_pattern, y_name, mask_name, default_values):
    # the mask pattern defines which agents to predict (true: predict agent)
    # therefore we mask the target data for those agents that are predicted
    # we also compute the union between the new mask and the previous mask
    data = mask_data(data, mask_pattern, y_name, default_values)
    data[mask_name] = data[mask_name] & mask_pattern[:, :, np.newaxis]
    data["autoreg_mask"] = th.ones_like(data[mask_name]) & mask_pattern[:, :, np.newaxis]
    return data


def create_fully_connected(n_nodes, n_groups=1, device=th.device('cpu')):
    return th.tensor(
        [
            [i + k * n_nodes, j + k * n_nodes]
            for k in range(n_groups)
            for i in range(n_nodes)
            for j in range(n_nodes)
            if i != j
        ], device=device
    ).T

conf_m_all = []


for i, train_data, test_data in get_cross_validations(
    data, n_cross_val, fraction_training
):
    model = AH_MODELS[model_name](
        default_values=default_values, autoregressive=autoregression, **model_args).to(
        th_device
    )

    batch_size = train_args["batch_size"]
    batch_edge_index = create_fully_connected(n_player, n_groups=batch_size)
    train_edge_index = create_fully_connected(n_player, n_groups=train_data['contribution'].shape[0])
    if test_data is not None:
        test_edge_index = create_fully_connected(n_player, n_groups=test_data['contribution'].shape[0])

    y_name = model_args["y_name"]

    optimizer = th.optim.Adam(model.parameters(), **optimizer_args)
    loss_fn = th.nn.CrossEntropyLoss(reduction="none")
    sum_loss = 0
    n_steps = 0

    for e in range(train_args["epochs"]):
        rec.set_labels(cv_split=i, epoch=e)
        model.train()
        for j, b_data in enumerate(batch_loader(train_data, batch_size)):
            optimizer.zero_grad()

            p_idx = th.randint(0, len(training_mask_pattern), (batch_size,))
            b_data = apply_mask_pattern(b_data, training_mask_pattern[p_idx], y_name, mask_name, default_values)
            batch_data = model.encode(b_data, mask=mask_name, edge_index=batch_edge_index, device=th_device)

            y_logit = model(batch_data).flatten(end_dim=-2)
            y_pred = y_logit.softmax(-1)
            y_true = batch_data["y_enc"].flatten(end_dim=-2)
            mask = batch_data["mask"].flatten()

            loss = (
                loss_fn(y_logit, y_true)
                + (y_pred * y_pred.log()).sum(-1) * train_args["l1_entropy"]
            )
            
            loss = (loss * mask).sum() / mask.sum()

            loss.backward(retain_graph=True)

            if train_args["clamp_grad"]:
                for param in model.parameters():
                    param.grad.data.clamp_(
                        -train_args["clamp_grad"], train_args["clamp_grad"]
                    )

            optimizer.step()
            sum_loss += loss.item()
            n_steps += 1

        last_epoch = e == (train_args["epochs"] - 1)

        if (e % train_args["eval_period"] == 0) or last_epoch:
            avg_loss = sum_loss / n_steps
            print(f"CV {i} | Epoch {e} | Loss {avg_loss}")

            rec.rec(value=avg_loss, set="train")

            # evalute on training data for all possible mask patterns
            for j, mask in enumerate(test_mask_pattern): 
                n_pred = mask.sum().item()
                _d = apply_mask_pattern(train_data, mask[np.newaxis], y_name, mask_name, default_values)
                _d = model.encode(_d, mask=mask_name, edge_index=train_edge_index, device=th_device)
                metrics = eval_model(model, _d)
                rec.rec_many(metrics, set="train", n_pred=n_pred, mask=j)
                
            if test_data is not None:
                # evalute on training data for all possible mask patterns
                for j, mask in enumerate(test_mask_pattern): 
                    n_pred = mask.sum().item()
                    _d = apply_mask_pattern(test_data, mask[np.newaxis], y_name, mask_name, default_values)
                    _d = model.encode(_d, mask=mask_name, edge_index=test_edge_index, device=th_device)
                    metrics = eval_model(model, _d)
                    rec.rec_many(metrics, set="test", n_pred=n_pred, mask=j)
                    # evalute on training data, shuffled features
                    for sf in shuffle_features:
                        _d = apply_mask_pattern(test_data, mask[np.newaxis], y_name, mask_name, default_values)
                        _d = shuffle_feature(_d, sf)
                        _d = model.encode(
                            _d, mask=mask_name, edge_index=test_edge_index, device=th_device
                        )
                        metrics = eval_model(model, _d)
                        rec.rec_many(metrics, set="test", shuffle_feature=sf, n_pred=n_pred, mask=j)
            sum_loss = 0
            n_steps = 0


    if test_data is not None:
        # compute confusion matrix
        for j, mask in enumerate(test_mask_pattern): 
            n_pred = mask.sum().item()
            _d = apply_mask_pattern(test_data, mask[np.newaxis], y_name, mask_name, default_values)
            _d = model.encode(_d, mask=mask_name, edge_index=test_edge_index, device=th_device)
            conf_m_all.append(create_confusion_matrix(
                model, _d, y_name=y_name, labels={**labels, 'n_pred': n_pred, 'mask': j, 'set': 'test', 'cv_split': i}))

    if i is None:
        model.save(model_path)

    if len(conf_m_all) > 0:
        conf_m = pd.concat(conf_m_all)
        conf_m.to_parquet(conf_path)
    rec.save(output_dir, labels, job_id=job_id)

/home/mpib/brinkmann/repros/algorithmic-institutions/.venv/lib/python3.9/site-packages/torch/autograd/__init__.py:200: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


CV None | Epoch 0 | Loss 3.2350445985794067
CV None | Epoch 20 | Loss 3.1439163366953533
CV None | Epoch 29 | Loss 3.025625710134153
